# ASHRAE - Great Energy Predictor III

## 1. Preprocessing

### 1.1 Install Necessary Packages

In [ ]:
!pip install feather-format

### 1.2 Import Packages

In [ ]:
import pandas as pd
import numpy as np
import gc
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict, cross_val_score
import math
import lightgbm as lgb

### 1.3 Load Data

#### 1.3.1 Convert Pandas Dataframe to Feather

We use a framework called feather for more efficient read and write operations in the future.

In [ ]:
# Read data...
#root = '/kaggle/input/ashrae-energy-prediction'
root = './ashrae-energy-prediction'

# Read from csv files
train_df = pd.read_csv(os.path.join(root, 'train.csv'))
weather_train_df = pd.read_csv(os.path.join(root, 'weather_train.csv'))
test_df = pd.read_csv(os.path.join(root, 'test.csv'))
weather_test_df = pd.read_csv(os.path.join(root, 'weather_test.csv'))
building_meta_df = pd.read_csv(os.path.join(root, 'building_metadata.csv'))
sample_submission = pd.read_csv(os.path.join(root, 'sample_submission.csv'))



In [ ]:
# Save data in feather format
train_df.to_feather('train.feather')
test_df.to_feather('test.feather')
weather_train_df.to_feather('weather_train.feather')
weather_test_df.to_feather('weather_test.feather')
building_meta_df.to_feather('building_metadata.feather')
sample_submission.to_feather('sample_submission.feather')

# Delete raw data from memory for better performance
del train_df
del test_df
del weather_train_df
del weather_test_df
del building_meta_df
del sample_submission
gc.collect()

#### 1.3.2 Load Training Data

In [ ]:
train = pd.read_feather('train.feather')
weather_train = pd.read_feather('weather_train.feather')
building_meta = pd.read_feather('building_metadata.feather')


### 1.4 Data Cleaning

#### 1.4.1 Remove Outliers

Building 1099 has abnormally higher metering readings than the other buildings.

Buildings with id <= 104 has meter readings 0 before 05/20/2016, which is anomaly low.


In [ ]:
# Remove outliers
train = train[train['building_id'] != 1099]
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

#### 1.4.2 Scale Square Feet

Since square_feet spans several order of magnitude(some are very big and some are very small, we take the log for better comparability.

In [ ]:
building_meta['square_feet'] =  np.log1p(building_meta['square_feet'])

#### 1.4.3 Handle Missing Values 

For accuracy and least interference with model fitting, we first group the data base on site_id, month, and day. And fill missing with the daily average in a site.

For features such as 'air_temperature' and 'dew_temperature', we can fill both directions because they follow a normal distribution. (We can fill backwards)

For 'cloud_coverage', 'sea_level_pressure', 'precip_depth_1_hr', however, we only fill forward because next observation is not indicative of prior.

In [ ]:
import datetime
def weather_pipeline(weather):
    
    # Count total number of hours in dataset
    time_format = "%Y-%m-%d %H:%M:%S"
    start = datetime.datetime.strptime(weather['timestamp'].min(),time_format)
    end = datetime.datetime.strptime(weather['timestamp'].max(),time_format)
    all_hours = int(((end - start).total_seconds()) / 3600 + 1)
    
    
    timestamps = [(end - datetime.timedelta(hours=x)).strftime(time_format) for x in range(all_hours)]
    
    
    # Add missing timestamps for each site
    for site_id in range(16):
        split_site_hours = np.array(weather[weather['site_id'] == site_id]['timestamp'])
        # Get missing hours
        missing_hours = pd.DataFrame(np.setdiff1d(timestamps, split_site_hours), columns=['timestamp'])
        missing_hours['site_id'] = site_id
        weather = pd.concat([weather, missing_hours])
        weather = weather.reset_index(drop=True) 
    

        
    weather["datetime"] = pd.to_datetime(weather["timestamp"])
    weather["day"] = weather["datetime"].dt.day
    weather["week"] = weather["datetime"].dt.week
    weather["month"] = weather["datetime"].dt.month
    
    weather = weather.set_index(['site_id','day','month'])
    
    
    normal = ['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']
    forward = ['cloud_coverage', 'sea_level_pressure', 'precip_depth_1_hr']
    
    # Fill both directions with mean
    for item in normal:
        feature_filler = pd.DataFrame(weather.groupby(['site_id','day','month'])[item].mean(), columns=[item])
        weather.update(feature_filler, overwrite=False)
    
    
    # Fill forward 
    for item in forward:
        feature_filler = weather.groupby(['site_id','day','month'])[item].mean()
        feature_filler = pd.DataFrame(feature_filler.fillna(method='ffill'),columns=[item])
        weather.update(feature_filler,overwrite=False)

    weather = weather.reset_index()
    weather = weather.drop(['datetime','day','week','month'],axis=1)
        
    return weather


In [ ]:
weather_train = weather_pipeline(weather_train)

In [ ]:
weather_train.info()

#### 1.4.4 Memory Reduction and Merge Data

In [ ]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train = reduce_mem_usage(train,use_float16=True)
building_meta = reduce_mem_usage(building_meta,use_float16=True)
weather_train = reduce_mem_usage(weather_train,use_float16=True)

train = train.merge(building_meta, left_on='building_id',right_on='building_id',how='left')
train = train.merge(weather_train, how='left', left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_train
del building_meta
gc.collect()

#### 1.4.5 Feature Engineering


In [ ]:
from sklearn.preprocessing import LabelEncoder
def FE_pipeline(data):
    
    # Sort by timestamp
    data.sort_values("timestamp")
    data.reset_index(drop=True)
    
    # Add more features
    data["timestamp"] = pd.to_datetime(data["timestamp"],format="%Y-%m-%d %H:%M:%S")
    data["hour"] = data["timestamp"].dt.hour
    data["day_of_week"] = data["timestamp"].dt.weekday
    
    
    # Remove Unused Columns
    feature_to_drop = ["timestamp",
                       "sea_level_pressure", 
                       "wind_direction", 
                       "wind_speed",
                       "year_built",
                       "floor_count"]
    data = data.drop(feature_to_drop, axis=1)
    gc.collect()
    
    # Encode Categorical Data
    labelencoder = LabelEncoder()
    data["primary_use"] = labelencoder.fit_transform(data["primary_use"])
    
    return data

In [ ]:
train = FE_pipeline(train)

#### 1.4.6 Fix Unit Error

In [ ]:
# Fix unit error 
train.loc[(train['site_id']==0) & (train['meter']==0), 'meter_reading'] = train.loc[(train['site_id']==0)&(train['meter']==0), 'meter_reading'] * 0.2931

In [ ]:
train[(train.site_id==0)&(train.meter==0)].describe()

## 2. Training and Evaluation

### 2.1 Scale Output

In [ ]:
y = np.log1p(train["meter_reading"])
X = train.drop('meter_reading', axis = 1)
del train
gc.collect()

### 2.2 Stratified Train-Test Split
Since we are training by site, we need to split evenly by site_id)

In [ ]:
# Split train and test with equal percentage of site id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True, stratify=X['site_id'])

### 2.3 Gradient Boosting, Divide & Conquer, and RMSE

- I.	Model Training
    - a.	Algorithm: Gradient Boosting
        - i.	Gradient Boosting is an ensemble machine learning technique that produces a prediction model by building an ensemble/sequence of weak prediction models (decision trees).
        - ii.	Unlike AdaBoost, which uses stumps, Gradient Boosting builds trees in a stage-wise manner. The general idea of the algorithm is as follow
            - 1.	Use the average of target variables as prediction for every instance;
            - 2.	Compute the Pseudo Residual = Observed – Prediction for each instance;
            - 3.	Build a decision tree using input variables with residuals as leaves at the lowest level; (If there are multiple residuals in one leaf, take the average of them)
            - 4.	Make a new prediction = Average Observed + Learning Rate x Residual for each instance; We use a Learning Rate to scale in order to prevent high variance(overfitting); Learning Rate results in a small step in the right direction. The model will perform better on testing dataset.
            - 5.	Compute the new Pseudo Residual = (Observed – (Average Observed + Sum of (Learning Rate x Previous Residuals))
            - 6.	Build a new tree based on new residuals.
            - 7.	Repeat until the maximum number of trees have been reached or additional trees fail to improve the fit.
            
        - iii. Pseudo Code
            - Input: Data {(xi, yi)} for i from 1 to n, and a differentiable loss function(in our case, the RMSLE)
            - Step 1: Initialize model with a constant value
                      
                $$F_0(x) = \operatorname*{argmin}_\gamma \sum_{i=1}^{n} L(y_i, \gamma)$$
            <br><br>
            - Step 2: For m = 1 to M where m is the index of tree and M is the maximum number of trees
                - (A) Compute:
$$r_{i,m} = -[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}]_{F(x) = F_{m-1}(x)}$$
                    
                     for i = 1, ... , n
                <br><br>
                - (B) Fit regression tree to the $r_{i,m}$ values and create terminal regions $R_{j,m}$, for j = 1...$J_m$ where j is the leaf index and $J_m$ is the number of leaves in tree m
                <br> <br>     
                - (C) For j = 1...$J_m$, compute: 
                    $$\gamma_{j, m} = \operatorname*{argmin}_\gamma \sum_{x_i\in{R_{i,j}}} L(y_i, F_{m-1}(x_i)+\gamma)$$
                <br> <br> 
                - (D) Update $$F_m(x) = F_{m-1}(x) + \nu \sum_{j=1}^{J_m} \gamma_{j,m} I(x\in{R_{j,m}})$$ where $\nu$ is the learning rate
                <br><br>
            
    - b. Framework: LightGBM
        - i. LightGBM was developed by Microsoft based on the idea of XGBoost. The difference between XGBoost and Naive Gradient Boosting which we described earlier is that XGBoost uses an objective Function
$$Obj(\theta) = L(\theta) + \Omega(\theta)$$
            where $L(\theta)$ is the loss function we previously described and $\Omega(\theta)$ is the regularization.
            <br>
            <br>
            Let's say we have $f_t(x) = w_q(x), w\in{R^T}$, q: ${\mathbb{R}}^d$ -> {1, 2, ... , Q} where w is the leaf weight of the tree and q is the structure of the tree, then
            $$\Omega(f_t) = \frac{1}{2} \times \lambda(\sum_{j=1}^{Q} {w_j}^{2}) + \alpha(\sum_{i=1}^{Q} |w_i|) + \gamma Q$$
            
            where $\lambda$ is XGBoost's lambda, $\sum_{j=1}^{Q} {w_j}^{2}$ is L1 penalty, $\alpha$ is XGboost's alpha, $\sum_{i=1}^{Q} |w_i|$ is L2 penalty, $\gamma$ is XGBoost's gamma, and Q is the number of leaves.
            
            <br>
            So after combining the loss and regularization, the formula becomes
$$ Obj(t) = \sum_{i=1}^{n} (g_i\times w_{q(x_i)} + \frac{h_i}{2} \times w_{q(x_i)}) + \frac{1}{2} \times \lambda(\sum_{j=1}^{Q} {w_j}^{2}) + \alpha(\sum_{i=1}^{Q} |w_i|) + \gamma Q$$

          $$= \sum_{j=1}^{Q} [(\sum_{i\in{I_j}} g_i \times w_{q(x_i)} + \frac{(\sum_{i\in{I_j}} h_i) + \lambda}{2} \times w_{q_(x_i)} ^ 2] + \gamma Q$$ 
          
          where $I_j = \{i|q(x_i) = k\}$ is a set of indices of points assigned to j-th leaf, $g_i$ and $h_i$ are the gradient(first-degree) and hessian(second-degree) of the loss function.
          <br>
          For simplification, let
          $$ G_j = \sum_{i\in{I_j}} g_i$$
          $$ H_i = \sum_{i\in_{I_j}} h_i$$
          <br>
          Then, the optimal weight for a fixed tree is 
          $$w_j^* = - \frac{G_j}{H_i+\lambda}$$
          <br>
          LightGBM grows the tree greedily by computing the gain
          $$ Gain = \frac{1}{2} [\frac{G_L^2}{H_L+\lambda} + \frac{G_R^2}{H_R+\lambda} - \frac{(G_L+G_R)}{H_L+H_R+\lambda}] - \lambda$$
          <br>
          where $\frac{G_L^2}{H_L+\lambda}$ is the left child score, $\frac{G_R^2}{H_R+\lambda}$ is the right child score,  $\frac{(G_L+G_R)}{H_L+H_R+\lambda}$ is the score if we do not split, and $\lambda$ is the complexity cost if we add a new split.
          <br>
          <br>
          LightGBM finds the best split point by iterating over sorted attributes and compute the gain.
          <br>
          <br>
      - ii. Some characteristics of LightGBM
      <br>
          - Tree is grown in Breadth-First fashion instead of Depth-First so that we can sort and traverse the data only once on each level
          <br>
          - Sorted features can be cached, so that we don't have to sort many times
          <br>
          - Each continuous feature is bucketed into discrete bins, and we iterate over number of bins instead of number of points
    
          




- c. Training Methodology: Divide and Conquer
    - We will split the data by site and train a separate model on each site. 
    - The reasons:
        - 1. There are time zone differences of different sites.
        - 2. The size of the data is very large and it is difficult to fit a single model that can generalize on multiple locations despite the advantages of gradient boosting.
        - 3. Decrease the training time 
<br>
<br>

- d. Evaluation Metric: Since we have already logged the output variable, we can just use RMSE as the evaluation metric which will produce RMSLE.





In [ ]:
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score
import math

def rmse(y_true, y_pred):
    """
    Returns the RMSE of prediction values
    """
    assert len(y_true) == len(y_pred)
    y_true = y_true.values
    terms_to_sum = 0
    for i, pred in enumerate(y_pred):
        prediction = y_pred[i] if y_pred[i] > 0 else 0 # Replace negative values with 0
        terms_to_sum += (prediction - y_true[i]) ** 2.0
    return ((terms_to_sum * (1.0/len(y_true))) ** 0.5)

scorer = make_scorer(rmse)

### 2.4 First Attempt
We will just use default parameters for now and plot the distribution of our prediction to see how the model performs.

In [ ]:
categorical_features = ["building_id", "site_id", "meter", "primary_use", "day_of_week"]

for site_id in range(16):
    print('Site:', site_id)
    reg = lgb.LGBMRegressor(n_threads=16, verbose=0)
    reg.fit(X_train[X_train.site_id==site_id], 
            y_train[X_train.site_id==site_id], 
            categorical_feature=categorical_features, 
            eval_metric='rmse', 
            verbose=False)
    
    y_pred = reg.predict(X_test[X_test.site_id==site_id])
    
    # Plot Distribution
    sns.distplot(y_pred, label='Prediction')
    sns.distplot(y_test[X_test.site_id==site_id], label='Ground Truth')
    plt.legend()
    plt.show()

Observation:
- Site 1, 2, 6, 7, 9, 10, 11, 13, 14, 15 all seem to have an abnormal amount of 0 meter_reading in ground truth which we fail to predict. 
- The model seems a little bit overfit. We are worried that it will not generalize well on the testing dataset which is twice as big as the training dataset.
- Therefore, We will tune the parameters next.

## 3. Hyperparameter Tuning

### 3.1 LightGBM parameters:
    - 1. learning_rate: Multiplication factor on each boosting iteration; (0, 1]
    - 2. n_estimators: Number of boosted tree to fit; Large value can decrease overfitting
    - 3. num_leaves: Maximum Number of Leaves; Large value can improve accuracy but expose to overfitting. Typical value is 2 ^ max_depth
    - 4. max_depth: Maximum depth of boosted tree; Too deep can cause overfitting
    - 5. min_data_in_leaf: Very important parameter to prevent overfitting
    - 6. bagging_fraction: Randomly select part of rows(instances); Can be used to prevent overfitting
    - 7. feature_fraction: Selecting part of the columns(features); Can be used to deal with the curse of dimensionality 
    - 8. lambda_l1: l1 regularization
    - 9. lambda_l2: l2 regularization

### 3.2 Hyperopt Bayesian Optimization

#### 3.2.1 Search Space

In [ ]:
from hyperopt import hp, tpe, fmin
lgb_space = {'boosting': hp.choice('booster', ['gbdt', 'dart']),
             'num_leaves': hp.choice('num_leaves', [32, 64, 128, 256, 512, 1024, 2048]),  # The number of leavers per tree
             'learning_rate': hp.quniform('learning_rate', 0.001, 0.05, 0.001), # The learning rate of boosted tree
             'max_depth': hp.quniform('max_depth', 5, 20, 1), # The maximum depths per tree(prevent overfitting)
             'feature_fraction': hp.quniform('feature_fraction', 0.5, 1.0, 0.1), # Percentage of features to use for tree
             'bagging_fraction': hp.quniform('bagging_fraction', 0.7, 1.0, 0.1), # Percentage of rows for tree
             'min_data_in_leaf': hp.quniform('min_data_in_leaf', 20, 100, 10), # Minimum data in leaf(prevent overfitting) 
             'lambda_l1' : hp.loguniform('lambda_l1', np.log(0.1), np.log(10)), # l1 regularization(prevent overfitting)
             'lambda_l2' : hp.loguniform('lambda_l2', np.log(0.1), np.log(10)) # l2 regularization(prevent overfitting)
             #'min_gain_to_split':hp.quniform('min_gain_to_split'0.0, 1.0, 0.1) # gamma
            }



In [ ]:
import multiprocessing
multiprocessing.cpu_count()
del train
del X_train
del X_test
del y_train
del y_test

In [ ]:
gc.collect()

#### 3.2.2 Objective Function

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings("ignore")

best_params = {} # Best parameter for each site

for i in range(16):
    def lgb_objective(params):
        params = {'boosting': params['boosting'],
                    'num_leaves': int(params['num_leaves']),
                  'learning_rate': params['learning_rate'],
                  'max_depth': int(params['max_depth']),
                  'feature_fraction': params['feature_fraction'],
                  'bagging_fraction': params['bagging_fraction'],
                  'min_data_in_leaf': int(params['min_data_in_leaf']),
                  'lambda_l1' : params['lambda_l1'],
                  'lambda_l2' : params['lambda_l2']}

        gbm_reg = lgb.LGBMRegressor(n_estimators=1000, 
                                    #early_stopping_round=50,
                                    n_threads=16, # Multithreading to speed up training
                                    #verbose=-1,
                                    **params) # Pass in the parameter space

        best_score = cross_val_score(gbm_reg, 
                                     X[X.site_id==i], 
                                     y[X.site_id==i], 
                                     fit_params={'verbose' : False,
                                                'categorical_feature' : categorical_features}, 
                                     scoring=scorer,
                                     cv=3 # 3 fold cross validation(67% training, 33% evaluation)
                                     ).mean()

        return best_score
    
    print("Tuning parameter for Site", i)
    
    best = fmin(fn=lgb_objective,
                    space=lgb_space,
                    max_evals=10,
                    rstate=np.random.RandomState(42),
                    algo=tpe.suggest)
    
    
    print("Best Parameter for Site", i, ": ", best)
    
    # Save best parameters
    best_params[i] = best;
    
    # Write parameters to file
    fname = "best_params_" + str(i) + ".txt"
    f = open(fname, "w+")
    f.write(str(best))
    f.close()
    gc.collect()
    

## 4. Generalization

### 4.1 Load test data

In [ ]:
test = pd.read_feather('test.feather')
weather_test = pd.read_feather('weather_test.feather')

In [ ]:
weather_test.head(5)

In [ ]:
test = reduce_mem_usage(test)
test.head(5)

### 4.2 Merge Building Data

In [ ]:
building_meta = pd.read_feather('building_metadata.feather')
building_meta = reduce_mem_usage(building_meta,use_float16=True)

In [ ]:
building_meta.head(5)

In [ ]:
test = test.merge(building_meta, left_on='building_id', right_on='building_id',how='left')

In [ ]:
test.head(5)

In [ ]:
del building_meta
gc.collect()

### 4.3 Fill Weather Data

In [ ]:
weather_test = weather_pipeline(weather_test)
weather_test = reduce_mem_usage(weather_test)
weather_test.head(5)

### 4.4 Merge Weather Data

In [ ]:
test = test.merge(weather_test, how='left',on=['timestamp','site_id'])
test.head(5)

In [ ]:
del weather_test
gc.collect()

### 4.5 Feature Engineering

In [ ]:
test = FE_pipeline(test)

In [ ]:
test.head(5)

In [ ]:
for i in range(16):
    best_params[i]['n_threads'] = 16
    best_params[i]['metric'] = 'rmse'
    best_params[i]['objective'] = 'regression'
    best_params[i]['boosting'] = 'gbdt'
    best_params[i]['max_depth'] = int(best_params[i]['max_depth'])
    best_params[i]['num_leaves'] = int(best_params[i]['num_leaves'])
    best_params[i]['min_data_in_leaf'] = int(best_params[i]['min_data_in_leaf'])
    print(best_params[i])

In [ ]:
# Dictionary to store the models
models = {}
for i in range(16):
    models[i] = []

# Dictionary to store evaluation results for later graphing
eval_results = {}
for i in range(16):
    eval_results[i] = {}

### 4.6 Fit Models by Site

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook as tqdm

# Make predictions based on sites
for site_id in tqdm(range(16), desc='site_id'):
    print("Training models for Site", site_id)
    
    # Create a mask filtering site_id 
    site_mask = (X.site_id == site_id)
    
    # Partition training data into 3 parts with equal percentage for each meter type
    skf = StratifiedKFold(n_splits=3, shuffle=True) # 3-fold
    fold = 0
    for train_indices, test_indices in skf.split(X[site_mask], X[site_mask]['meter']):
        print("Fold:", fold)
        fold += 1
        
        X_train = X[site_mask].iloc[train_indices]
        y_train = y[site_mask].iloc[train_indices]

        X_test = X[site_mask].iloc[test_indices]
        y_test = y[site_mask].iloc[test_indices]

        d_train = lgb.Dataset(X_train, 
                              label=y_train, 
                              categorical_feature=categorical_features, 
                              free_raw_data=False)
        
        d_test = lgb.Dataset(X_test, 
                             label=y_test, 
                             categorical_feature=categorical_features, 
                             free_raw_data=False)

        
        model = lgb.train(best_params[site_id], # The optimal parameters for each site 
                          train_set=d_train, 
                          num_boost_round=1000, 
                          valid_sets=[d_test, d_train],
                          valid_names = ['validation', 'train'],
                          verbose_eval=0, 
                          evals_result = eval_results[site_id][fold],
                          early_stopping_rounds=50)
        
        models[site_id].append(model)
        
        del X_train, y_train, X_test, y_test, d_train, d_test
        gc.collect()
    
    

### 4.7 Plot learning curve

In [ ]:
import matplotlib.pyplot as plt
for site_id in range(16):
    print('Learning Curve for Site', site_id)
    ax = lgb.plot_metric(eval_results[site_id], metric='rmse')
    plt.show()

### 4.8 Prediction

In [ ]:
input_cols = test.columns.tolist()

In [ ]:
input_cols.remove('row_id')
input_cols

In [ ]:
test['meter'] = test['meter'].astype('category')
test['meter'].dtype

In [ ]:
test_sites = []
for site_id in tqdm(range(16), desc="site_id"):
    test_site = test[test.site_id==site_id]
    row_ids_site = test_site.row_id

    test_site = test_site[input_cols]
    y_pred = np.zeros(test_site.shape[0])

    print("Predicting meter reading for Site", site_id)    
    for fold in range(3):
        model = models[site_id][fold]
        # We take the average of the prediction result of the 3 models
        y_pred += model.predict(test_site, 
                                          num_iteration=model.best_iteration) / 3
        gc.collect()
        
    df_test_site = pd.DataFrame({"row_id": row_ids_site, 
                                 "meter_reading": y_pred})
    # convert unit
    if(site_id==0):
        df_test_site[df_test_site.meter_reading==0] *= 3.4118
    df_test_sites.append(df_test_site)
    
    print("Prediction for site_id", site_id, "completed\n")
    gc.collect()

## 5. Submission

In [ ]:
# Concatenate
submit = pd.concat(df_test_sites)
submit.meter_reading = np.clip(np.expm1(submit.meter_reading), 0, a_max=None)
submit.to_csv("submission.csv", index=False)